#Test

In [1]:
# !pip install lightning


import torch
from torch.utils.data import *
import torchvision
import torchvision.transforms as transforms
import lightning as L


class LightningDataModule(L.LightningDataModule):
    def __init__(self, data_dir: str = "./data"):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        self.batch_size = 4
        self.dims = (3, 32, 32)
        self.num_classes = 10

    def prepare_data(self):
        # download
        torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=self.transform)
        torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=self.transform)

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            cifar_full = torchvision.datasets.CIFAR10(self.data_dir, train=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.cifar_test = torchvision.datasets.CIFAR10(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size,
                          shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size,
                          shuffle=False, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size,
                          shuffle=False, num_workers=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 14.2 MB/s eta 0:00:00


In [2]:
# from load_data import *

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics import Accuracy


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class LightningModule(L.LightningModule):
    def __init__(self, lr=0.05):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net()
        self.train_accuracy = Accuracy(task="multiclass", num_classes=10)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss)
        self.log('train_accuracy', accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        accuracy = self.val_accuracy(logits, y) # (logits.argmax(dim=1) == y).float().mean() # = self.val_accuracy(logits, y)
        self.log('val_loss', loss)
        self.log('val_accuracy', accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        accuracy = self.val_accuracy(logits, y)  # (logits.argmax(dim=1) == y).float().mean() # = self.val_accuracy(logits, y)
        self.log('test_loss', loss)
        self.log('test_accuracy', accuracy, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams.lr, momentum=0.9)
        return optimizer

In [7]:
!pip install wandb

import wandb
wandb.login()
# wandb.init(project="proj_pytorch")
# wandb.finish()

# !pip list

wandb: Currently logged in as: proxod3. Use `wandb login --relogin` to force relogin


True

In [9]:
# from nn_model import *

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar


L.seed_everything(1)

#dm - data_module
dm = LightningDataModule()
dm.prepare_data()
dm.setup()

model = LightningModule(lr=0.001)

wandb_logger = WandbLogger(log_model="all", project="proj_pytorch", name="magic-dragon-1")

# callbacks = [
#     ModelCheckpoint(
#         dirpath = "checkpoints",
#         every_n_train_steps=100,
#     ),
# ]


trainer = L.Trainer(
    max_epochs=1,
    accelerator="auto",
    devices="auto",
    logger=wandb_logger,
    # callbacks=callbacks,
    # callbacks=[RichProgressBar()],
    # callbacks=EarlyStopping('val_loss', patience=7),
)

trainer.fit(model, dm.train_dataloader(), dm.val_dataloader())

trainer.test(model, dm.test_dataloader())



# sweep_config = {
#     'method': 'random'
# }

# metric = {
#     'name': 'loss',
#     'goal': 'minimize'
#     }

# sweep_config['metric'] = metric

# parameters_dict = {
#     'optimizer': {
#         'values': ['sgd']
#         },
#     }

# sweep_config['parameters'] = parameters_dict

# parameters_dict.update({
#     'epochs': {
#         'value': 1}
#     })


# !pip install wandb -Uq
# import wandb

# wandb.login()

# !wandb sweep config.yaml


# import pprint

# pprint.pprint(sweep_config)

# sweep_id = wandb.sweep(sweep_config)

# !wandb agent sweep_id

# wandb.agent(sweep_id)
# # !wandb sweep --project proj

INFO: Seed set to 1
INFO:lightning.fabric.utilities.seed:Seed set to 1


Files already downloaded and verified
Files already downloaded and verified


ModuleNotFoundError: ignored